In [23]:
# FM
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.metrics import MeanAbsoluteError
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

# GPU 확인
tf.config.list_physical_devices('GPU')

# 자료형 선언
tf.keras.backend.set_floatx('float32')


In [16]:

#### 마스터 데이터(상호 작용)
#### 상호 작용(transaction) 마스터 데이터를 불러옴
masterdf = pd.read_csv('./data/Transactions.csv')
masterdf.head()

### 데이터 정리 및 표준화를 위해 데이터 열 명칭 변경
### 표준화는 병합의 용의성을 위해 열 명칭을 정렬
masterdf.columns = ['Transaction ID', 'Customer ID', 'Transaction Date', 'Prod Subcat Code',
       'Prod Cat Code', 'Qty', 'Rate', 'Tax', 'Total Amt', 'Store Type']

###  상점 코드 타입을 숫자형으로 변경하여 새 열에 저장
masterdf['Store Type Code'] = pd.factorize(masterdf['Store Type'])[0]
masterdf.head(5)

### quantity와 based price에서 총 순 매출액(Net sales) 계산 (도시마다의 세금이 다를 수 있어 세금 제외)
masterdf['Net Sales'] = masterdf['Qty'] * masterdf['Rate']

### category, subcategory, store type을 이용하여 고유한 material 표시기를 생성
### 다른 sku는 다른 상점 유형에 판매된다고 가정
masterdf['Material'] = masterdf['Prod Cat Code'].astype(str) + '-' + masterdf['Prod Subcat Code'].astype(str) + '-' + masterdf['Store Type'].astype(str)

masterdf[['Customer ID','Material','Net Sales']]

scaler = MinMaxScaler()
grade = scaler.fit_transform(masterdf[['Net Sales']])

grade = pd.DataFrame(grade,columns=['grade'])
cust = pd.get_dummies(masterdf['Customer ID'],prefix='cust')
item = pd.get_dummies(masterdf['Material'],prefix='item')

df = pd.concat([cust,item,grade], axis=1)

In [17]:
# 데이터 로드
X, Y = df[[i for i in df.columns if i!='grade']].to_numpy(), df[['grade']].to_numpy()
X = X[:23050,:]
Y = Y[:23050,:]

n = X.shape[0]
p = X.shape[1]

k = 10
batch_size = 8
epochs = 10

In [18]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [19]:
Y

array([[0.51009421],
       [0.99798116],
       [0.20349933],
       ...,
       [0.11009421],
       [0.16069987],
       [0.08169583]])

In [20]:
class FM(tf.keras.Model):
    def __init__(self):
        super(FM, self).__init__()

        # 모델의 파라미터 정의
        self.w_0 = tf.Variable([0.0])
        self.w = tf.Variable(tf.zeros([p]))
        self.V = tf.Variable(tf.random.normal(shape=(p, k)))

    def call(self, inputs):
        linear_terms = tf.reduce_sum(tf.math.multiply(self.w, inputs), axis=1)

        interactions = 0.5 * tf.reduce_sum(
            tf.math.pow(tf.matmul(inputs, self.V), 2)
            - tf.matmul(tf.math.pow(inputs, 2), tf.math.pow(self.V, 2)),
            1,
            keepdims=False
        )

        y_hat = tf.math.sigmoid(self.w_0 + linear_terms + interactions)

        return y_hat

In [21]:
# Forward
def train_on_batch(model, optimizer, accuracy, inputs, targets):
    with tf.GradientTape() as tape:
        y_pred = model(inputs)
        loss = tf.keras.losses.mean_absolute_error(y_true=targets, y_pred=y_pred)
    
    # loss를 모델의 파라미터로 편미분하여 gradients를 구한다.
    grads = tape.gradient(target=loss, sources=model.trainable_variables)

    # apply_gradients()를 통해 processed gradients를 적용한다.
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # accuracy: update할 때마다 정확도는 누적되어 계산된다.
    accuracy.update_state(targets, y_pred)

    return loss


# 반복 학습 함수
# def train(epochs):
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

train_ds = tf.data.Dataset.from_tensor_slices(
    (tf.cast(X_train, tf.float32), tf.cast(Y_train, tf.float32))).shuffle(500).batch(8)

test_ds = tf.data.Dataset.from_tensor_slices(
    (tf.cast(X_test, tf.float32), tf.cast(Y_test, tf.float32))).shuffle(200).batch(8)

model = FM()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
accuracy = MeanAbsoluteError()
loss_history = []

for i in range(50):
    for x, y in train_ds:
        loss = train_on_batch(model, optimizer, accuracy, x, y)
        loss_history.append(loss)

    if i % 2== 0:
        print("스텝 {:03d}에서 누적 평균 손실: {:.4f}".format(i, np.mean(loss_history)))
        print("스텝 {:03d}에서 누적 정확도: {:.4f}".format(i, accuracy.result().numpy()))

스텝 000에서 누적 평균 손실: 0.3201
스텝 000에서 누적 정확도: 0.1651
스텝 002에서 누적 평균 손실: 0.3033
스텝 002에서 누적 정확도: 0.1504
스텝 004에서 누적 평균 손실: 0.2986
스텝 004에서 누적 정확도: 0.1463
스텝 006에서 누적 평균 손실: 0.2959
스텝 006에서 누적 정확도: 0.1439
스텝 008에서 누적 평균 손실: 0.2937
스텝 008에서 누적 정확도: 0.1421
스텝 010에서 누적 평균 손실: 0.2919
스텝 010에서 누적 정확도: 0.1405
스텝 012에서 누적 평균 손실: 0.2902
스텝 012에서 누적 정확도: 0.1390
스텝 014에서 누적 평균 손실: 0.2885
스텝 014에서 누적 정확도: 0.1376
스텝 016에서 누적 평균 손실: 0.2869
스텝 016에서 누적 정확도: 0.1363
스텝 018에서 누적 평균 손실: 0.2853
스텝 018에서 누적 정확도: 0.1349
스텝 020에서 누적 평균 손실: 0.2836
스텝 020에서 누적 정확도: 0.1335
스텝 022에서 누적 평균 손실: 0.2818
스텝 022에서 누적 정확도: 0.1321
스텝 024에서 누적 평균 손실: 0.2800
스텝 024에서 누적 정확도: 0.1306
스텝 026에서 누적 평균 손실: 0.2782
스텝 026에서 누적 정확도: 0.1291
스텝 028에서 누적 평균 손실: 0.2762
스텝 028에서 누적 정확도: 0.1276
스텝 030에서 누적 평균 손실: 0.2743
스텝 030에서 누적 정확도: 0.1260
스텝 032에서 누적 평균 손실: 0.2722
스텝 032에서 누적 정확도: 0.1243
스텝 034에서 누적 평균 손실: 0.2702
스텝 034에서 누적 정확도: 0.1227
스텝 036에서 누적 평균 손실: 0.2680
스텝 036에서 누적 정확도: 0.1210
스텝 038에서 누적 평균 손실: 0.2659
스텝 038에서 누적 정확도: 0.1193


In [24]:
test_accuracy = MeanAbsoluteError()
for x, y in test_ds:
    y_pred = model(x)
    test_accuracy.update_state(y, y_pred)

print("테스트 정확도: {:.4f}".format(test_accuracy.result().numpy()))

테스트 정확도: 0.2154
